In [1]:
import urllib.request, json
import pandas as pd
import folium

In [2]:
with urllib.request.urlopen("https://raw.githubusercontent.com/isellsoap/deutschlandGeoJSON/main/2_bundeslaender/4_niedrig.geo.json") as url:
    data = json.load(url)

In [4]:
for i in range(16):
    print(data["features"][i]["properties"]["name"])

Baden-Württemberg
Bayern
Berlin
Brandenburg
Bremen
Hamburg
Hessen
Mecklenburg-Vorpommern
Niedersachsen
Nordrhein-Westfalen
Rheinland-Pfalz
Saarland
Sachsen-Anhalt
Sachsen
Schleswig-Holstein
Thüringen


In [5]:
def main():
    json_data = get_json("https://raw.githubusercontent.com/isellsoap/deutschlandGeoJSON/main/2_bundeslaender/4_niedrig.geo.json")
    federal_states = extract_state_names_from_json(json_data)
    dataframe = create_df_temp_change(federal_states)
    map = create_map_with_tooltips(dataframe, json_data)

    map.save("Temp_Change_Germany.html")


def get_json(url_json):
    with urllib.request.urlopen(url_json) as url:
        return json.load(url)


def extract_state_names_from_json(json):
    list_federal_states = list()

    for i in range(len(json["features"])):
        list_federal_states.append(json["features"][i]["properties"]["name"])

    return list_federal_states


def create_df_temp_change(list_with_federal_states):

    df_main = pd.DataFrame(columns=["State", "Temp_Change"])

    for federal_state in list_with_federal_states:

        path = "./data/federal_states/" + federal_state + ".txt"
        df = pd.read_csv(path)

        df.columns = [c_name.strip() for c_name in df.columns.tolist()]

        df["DATE"] = pd.to_datetime(df["DATE"], format="%Y%m%d")
        df.set_index("DATE", inplace=True)

        df.drop(["SOUID", "Q_TG"], axis=1, inplace=True)

        df.TG = df.TG / 10

        df["year"] = df.index.year

        filter = (df["year"] >= 1970) & (df["year"] < 2022)
        df = df[filter]

        df_grouped = df.groupby("year")["TG"].mean()

        beginning_values = list()
        last_values = list()

        beginning_values.extend(df_grouped.iloc[:5,].to_list())
        last_values.extend(df_grouped.iloc[-5:,].to_list())

        start = sum(beginning_values) / len(beginning_values)
        end = sum(last_values) / len(last_values)

        new_row = pd.Series({'State': federal_state, 'Temp_Change': round((end - start),2)})

        df_main = pd.concat([df_main, new_row.to_frame().T], ignore_index=True)

    return df_main


def create_map_with_tooltips(df, data):
    m = folium.Map(location=[51.3, 10], zoom_start=6)

    choropleth = folium.Choropleth(
        geo_data=data,
        data=df,
        columns=["State", "Temp_Change"],
        key_on='feature.properties.name',
        fill_color='RdPu',
        fill_opacity=0.7,
        line_opacity=0.5,
        legend_name='Avg Temp-Change 1970 - 75 / 2016 - 21',
        smooth_factor=0
    ).add_to(m)

    tooltip_text = list()

    for i,j in zip(df.iloc[:,0], df.iloc[:,1]):
        tooltip_text.append(i + " : " + str(j) + "°C")

    for idx in range(len(tooltip_text)):
        data['features'][idx]['properties']['tooltip1'] = tooltip_text[idx]

    choropleth.geojson.add_child(
        folium.features.GeoJsonTooltip(['tooltip1'], labels=False)
    )

    return m


if __name__ == "__main__":
    main()